<a href="https://colab.research.google.com/github/nlp2bok/fintech_/blob/lmh_dev/Data_Preprocessing_MPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Preprocessing

##. 1. 금통위 의사록(MPB Analysis Report)

1. MPB 데이터 병합

 160개의 금통위 의사록을 하나의 csv파일에 넣는다.
  파일명에서 rDate추출, 파일 read를 통한 내용 추출('작성일자','내용')
2. 다양한 Tokenized 방식을 통해 전처리를 실행한다.

 Komoran, Hannanum,  Okt, Kkma, Mecab => Mecab사용
 
 -> Mecab install 필요
3. Token에 한글 태그를 부착한다.
4. 불용어 처리를 실행한다.
5. csv파일에 preprocessing data 컬럼을 추가한다(preData)

In [0]:
import pandas as pd
import numpy as np
import os
import csv

##Step 1. 전체 파일을 하나의 파일로 합치기

In [0]:
#print(dirpath) #현재경로 출력
workDIr = os.path.abspath('/Users/minhyeon/Desktop/Data_Preprocessing_MPB/MPB_Contents/')
cnt = 0

def fileopen(path):
    f = open(path,'r', encoding='utf-8')
    line = f.read()
    f.close()
    
    return line 

with open('MPB_File.csv', 'w', encoding='utf-8', newline='') as cvsFile :
    wr = csv.writer(cvsFile)

    for dirpath, dirnames, filenames in os.walk(workDIr):
        for filename in filenames:
            #파일 컨텐츠 읽기
            path = dirpath+'/'+filename
            fType = filename[-4:]
            try:
                if fType == ".txt" : 
                    text = fileopen(path)
                else :
                    continue
                #파일 타입 체크
            except Exception as ex:
                print(filename + '파일 read 에러가발생 했습니다', ex)


            try:
                rDate = filename[:4] #Report Date, String형
                #print(cnt)
                cnt += 1
                wr.writerow([rDate,text])
            except Exception as ex:
                print(filename + '에러가 발생 했습니다', ex)
    
cvsFile.close()

## Step2. Tokenized 실행

##2-1. 파일 읽어오기

In [0]:
#csv파일을 읽어오고 column명을 설정해준다.
mydata = pd.read_csv("MPB_File.csv",names=['rDate','MPBcontent'],encoding="utf-8")

In [0]:
#읽어온 데이터를 DataFrame형식으로 담는다.
df = pd.DataFrame(mydata)

In [0]:
#mycontent에 전처리할 데이터만 담는다.
mycontent = df['MPBcontent']
print(len(mycontent))

160


## 2-2. 다양한 방식의 Tokenized 활용해보기

기사별 Tokenized한 값을 komoran_tokens, hannanum_tokens, okt_tokens, kkma_tokens, Mecab_tokens로 각각 저장한 후 결과를 확인해본다.

-> 채권보고서에서 Mecab성능이 가장 좋은 것을 확인하였음

In [0]:
#MeCab 토큰화
from ekonlpy.tag import Mecab
mecab = Mecab()
mecab_tokens = []
for i in range(len(mycontent)):
    try:
        #print(mecab.pos(mycontent[i]))
        #break
        mecab_tokens.append(mecab.morphs(mycontent[i]))
        #print(i)
    except:
        #이상 데이터 제외
        print(str(i)+"error")

##Step3. Mecab을 사용하여 한글 태그 부착
가장 성능이 좋은 Mecab을 활용하기로 하였음.

한글 태그 부착은 .pos를 사용

In [0]:
#MeCab 토큰화(ekonlpy를 통한 한글 태그 부착 -> ekonlpy 설치 필요)
from ekonlpy.tag import Mecab
mecab = Mecab()
mecab_tokens = []
for i in range(len(mycontent)):
    try:
        mecab_tokens.append(mecab.pos(mycontent[i]))
        #print(i)
    except:
        #이상 데이터 제외
        print(str(i)+'error')
        #이상한 데이터에는 빈 값을 넣어준다.
        mecab_tokens.append(' ')

In [0]:
#한글태그 부착확인
print(mecab_tokens[0][1])
print(mecab_tokens[0])

#데이터 개수 확인
print(len(mycontent))
print(len(mecab_tokens))

('년', 'NNBC')
160
160


##Step4. 불용어 처리

(1) stopInclude에 포함된 품사만 남긴다.

(2) stopWord는 불필요한 단어를 수동으로 제거

mecab-ko-dic 품사태그

명사(NNG), 형용사(VA,VAX), 부사(MAG), 동사(VA)만 추출


In [0]:
##명사(NNG), 형용사(VA,VAX), 부사(MAG), 동사(VA)
includePos = ['NNG','VA','VAX','MAG','VA']
stopWord = ["덤프데이터"]
result =[]
preresult = []
cnt = 0
#1493개의 mecab_tokens loop
for mecab_token in mecab_tokens :
    #print(mecab_token)
    #mecab_token(i)번째의 (word/형태소) 데이터에 접근 => wordData = ('단어','품사')
    cnt+=1
    #print(cnt)
    for wordData in mecab_token:
        #('단어','품사')중 '형태소' 데이터가 stopInclude에 포함, '단어' 데이터가 stopWord에 포함되지 않으면 저장한다.
        try:
            if wordData[1] in includePos:
                if wordData[0] not in stopWord:
                    preresult.append(wordData[0])
        except:
            continue
    result.append(','.join(preresult))
    preresult = []

##Step5. 전처리된 데이터를 기존 파일에 추가한다.

In [0]:
#print(len(result))
df['preData'] = result
print(df['preData'])

In [0]:

df.to_csv("/Users/minhyeon/Desktop/Data_Preprocessing_MPB/MPB_final.csv", header=True, index=False)

In [0]:
text = pd.read_csv("MPB_final.csv",encoding="utf-8")

In [0]:
print(text['preData'][0])